In [2]:
import sys
import os

# Append the directory of the module to the Python path
sys.path.append("/Users/sherryyang/Projects/super-segger-toolkit/superseggertoolkit")

In [15]:
from cell import Cell
from link_composer import LinkComposer
import cells_extractor 
import visualizer
import link_algorithm
from cell_event import CellEvent, CellDefine, Cell


In [4]:
import networkx as nx
from queue import Queue

In [5]:
composer = LinkComposer.read_folder(mask_folder="/Users/sherryyang/Documents/wiggins-lab/data/f/masks/*.png", phase_folder="/Users/sherryyang/Documents/wiggins-lab/data/f/phase/*.tif")

Frame:7, Mask label:1. Pixels number = 2. cannot make polygon. A linearring requires at least 4 coordinates.
Frame:11, Mask label:46. Pixels number = 1. cannot make polygon. A linearring requires at least 4 coordinates.
Frame:13, Mask label:1. Pixels number = 1. cannot make polygon. A linearring requires at least 4 coordinates.
Frame:22, Mask label:26. Pixels number = 2. cannot make polygon. A linearring requires at least 4 coordinates.


In [6]:
import pandas as pd

G = composer.make_new_dircted_graph()

excel_file = pd.ExcelFile("/Users/sherryyang/Documents/wiggins-lab/data/manuelly_linked_new.xlsx")
sheet_names = excel_file.sheet_names

In [7]:
import numpy as np

for frame_start in range(len(sheet_names)):
    frame_end = frame_start + 1
    time_sheet = sheet_names[frame_start]
    df = excel_file.parse(time_sheet)
    df = df.astype(str)
    [source, target] = df.columns
    for index, row in df.iterrows():
        df = df.dropna()
        df = df.astype(float).astype(int)
        print(frame_start, row[source], frame_end, row[target])
        composer.link(G, Cell(frame_start, int(row[source])), Cell(frame_end, int(row[target])))

0 3 1 3
0 5 1 5
0 6 1 6
0 9 1 10
0 4 1 4
0 2 1 2
0 8 1 9
0 10 1 11
0 11 1 12
0 15 1 16
0 22 1 23
0 23 1 25
0 19 1 20
0 17 1 18
0 14 1 15
0 16 1 17
0 26 1 29
0 25 1 28
0 34 1 38
0 32 1 35
0 39 1 42
0 30 1 33
0 24 1 26
0 33 1 37
0 27 1 30
0 31 1 36
0 28 1 32
0 41 1 45
0 21 1 24
0 29 1 31
1 2 2 2
1 3 2 3
1 4 2 4
1 5 2 5
1 6 2 6
1 9 2 8
1 10 2 9
1 11 2 10
1 12 2 11
1 15 2 14
1 16 2 15
1 17 2 16
1 18 2 17
1 20 2 20
1 23 2 21
1 24 2 22
1 25 2 23
1 26 2 24
1 28 2 25
1 29 2 26
1 30 2 27
1 31 2 28
1 32 2 29
1 33 2 30
1 35 2 31
1 36 2 32
1 37 2 33
1 38 2 34
1 42 2 38
1 45 2 40
2 3 3 3
2 5 3 5
2 6 3 6
2 9 3 9
2 4 3 4
2 2 3 2
2 8 3 8
2 10 3 10
2 11 3 11
2 15 3 15
2 21 3 21
2 23 3 22
2 20 3 19
2 17 3 17
2 14 3 14
2 16 3 16
2 26 3 25
2 25 3 24
2 34 3 33
2 31 3 30
2 38 3 37
2 30 3 29
2 24 3 23
2 33 3 32
2 27 3 26
2 32 3 31
2 29 3 28
2 40 3 39
2 28 3 27
2 22 3 20
3 3 4 3
3 5 4 5
3 6 4 6
3 9 4 10
3 4 4 4
3 2 4 2
3 8 4 8
3 10 4 11
3 11 4 12
3 15 4 16
3 21 4 22
3 22 4 24
3 19 4 20
3 17 4 18
3 14 4 15
3 1

In [8]:
G_manual = G

In [9]:
def cut_graph(G):
    for u, v in G.edges():
        G[u][v]['reliability'] = sys.maxsize

    for node in G.nodes():
        edges_with_weights = G.edges(node, data='weight')
        sorted_edges = sorted(edges_with_weights, key=lambda x: x[2])
        for i, (u, v, weight) in enumerate(sorted_edges):
            G[u][v]['reliability'] = min(i, G[u][v]['reliability'])

    edges_with_rel=  G.edges(data='reliability')
    sorted_edges = sorted(edges_with_rel, key=lambda x: x[2], reverse=True)
    index = 0

    while nx.is_connected(G):
        u,v, realiability_target = sorted_edges[index]
        realiability = realiability_target
        while realiability == realiability_target:
            index += 1
            G.remove_edge(u, v)
            u,v, realiability_target = sorted_edges[index]

    return G

def link_subtree(G):
    all_edges = set(G.edges(data='weight'))
    test_G = nx.Graph()
    edges = set()
    for u, v, weight in all_edges:
        if G.degree(u) == 1 or G.degree(v) == 1:
            test_G.add_edge(u,v, weight = 0)
        else:
            edges.add((u,v, weight))

    edges = list(edges)
    test_G.add_nodes_from(G.nodes())

    min_cost = sys.maxsize
    min_tree = nx.Graph()

    min_cost, min_tree = link_subtree_helper(0, edges, test_G, min_cost, min_tree)
    return min_cost, min_tree



def link_subtree_helper(index, edges, test_G, min_cost, min_tree):
    if index == len(edges):
        cost = sum(weight for _, _, weight in test_G.edges(data='weight'))
        cost += len(list(nx.isolates(test_G))) * 10
        if cost < min_cost:
            min_cost = cost
            min_tree = test_G.copy()
        return min_cost, min_tree

    u, v, weight = edges[index]

    min_cost, min_tree = link_subtree_helper(index + 1, edges, test_G, min_cost, min_tree)
    if (test_G.degree(u) <= 1) or (test_G.degree(v) <= 1):
        test_G.add_edge(u, v, weight=weight)
        min_cost, min_tree = link_subtree_helper(index + 1, edges, test_G, min_cost, min_tree)
        test_G.remove_edge(u, v)

    return min_cost, min_tree

In [10]:
tree_set = set()

for frame in range(1,composer.frame_num):

    dict = composer.cells_frame_dict
    sorted_frame = sorted(dict)

    source_cells = list(dict[sorted_frame[frame-1]])
    target_cells =  list(dict[sorted_frame[frame]])

    source_cells.sort()
    target_cells.sort()

    source_dim = len(source_cells)
    target_dim = len(target_cells)

    sub_G = nx.Graph()
    for i in range(source_dim):
        for j in range(target_dim):
            source_cell = source_cells[i]
            target_cell = target_cells[j]

            intersection_area = source_cell.polygon.intersection(target_cell.polygon).area
            if intersection_area > 0:
                union_area = source_cell.polygon.union(target_cell.polygon).area
                weight = -1 * intersection_area /  union_area  + 1
                sub_G.add_edge(source_cell, target_cell, weight=weight)

    
    queue = Queue()
    queue.put(sub_G)

    while not queue.empty():
        sub_G = queue.get()
        components = nx.connected_components(sub_G)
        subgraphs = []
        for nodes in components:
            subgraph = sub_G.subgraph(nodes).copy()
            if len(subgraph.edges()) < 20:
                min_cost, min_tree = link_subtree(subgraph)
                tree_set.add(min_tree)
            else:
                subgraph = cut_graph(subgraph)
                queue.put(subgraph)


G_new = composer.make_new_dircted_graph()
for graph in tree_set:
    for u, v in graph.edges():
        if u < v:
            G_new.add_edge(u,v)
        else:
            G_new.add_edge(v,u)

In [11]:
spots_filename = "/Users/sherryyang/Documents/wiggins-lab/data/f/spots_filamented.csv"
edge_filename = "/Users/sherryyang/Documents/wiggins-lab/data/f/track_filamented.csv"
G_over = composer.get_trackmate_linking_result(spots_filename, edge_filename, UNIT_CONVERT_COEFF= 100)

/Users/sherryyang/Projects/super-segger-toolkit/superseggertoolkit/link_composer.py:243: UserWarning: Trackmate cell:3732 match back to mask is inaccute, assigned to the nearest cell.
  warnings.warn(f"Trackmate cell:{cell_id} match back to mask is inaccute, assigned to the nearest cell.")
/Users/sherryyang/Projects/super-segger-toolkit/superseggertoolkit/link_composer.py:243: UserWarning: Trackmate cell:3951 match back to mask is inaccute, assigned to the nearest cell.
  warnings.warn(f"Trackmate cell:{cell_id} match back to mask is inaccute, assigned to the nearest cell.")
/Users/sherryyang/Projects/super-segger-toolkit/superseggertoolkit/link_composer.py:243: UserWarning: Trackmate cell:3978 match back to mask is inaccute, assigned to the nearest cell.
  warnings.warn(f"Trackmate cell:{cell_id} match back to mask is inaccute, assigned to the nearest cell.")
/Users/sherryyang/Projects/super-segger-toolkit/superseggertoolkit/link_composer.py:243: UserWarning: Trackmate cell:4054 match

In [12]:
spots_filename = "/Users/sherryyang/Documents/wiggins-lab/data/f/spots_lap.csv"
edge_filename = "/Users/sherryyang/Documents/wiggins-lab/data/f/edges_lap.csv"
G_lap = composer.get_trackmate_linking_result(spots_filename, edge_filename, UNIT_CONVERT_COEFF= 100)

/Users/sherryyang/Projects/super-segger-toolkit/superseggertoolkit/link_composer.py:243: UserWarning: Trackmate cell:5519 match back to mask is inaccute, assigned to the nearest cell.
  warnings.warn(f"Trackmate cell:{cell_id} match back to mask is inaccute, assigned to the nearest cell.")
/Users/sherryyang/Projects/super-segger-toolkit/superseggertoolkit/link_composer.py:243: UserWarning: Trackmate cell:5292 match back to mask is inaccute, assigned to the nearest cell.
  warnings.warn(f"Trackmate cell:{cell_id} match back to mask is inaccute, assigned to the nearest cell.")
/Users/sherryyang/Projects/super-segger-toolkit/superseggertoolkit/link_composer.py:243: UserWarning: Trackmate cell:5566 match back to mask is inaccute, assigned to the nearest cell.
  warnings.warn(f"Trackmate cell:{cell_id} match back to mask is inaccute, assigned to the nearest cell.")
/Users/sherryyang/Projects/super-segger-toolkit/superseggertoolkit/link_composer.py:243: UserWarning: Trackmate cell:5598 match

In [13]:
G_extra_edge = set(G.edges()).difference(set(G_manual.edges()))
warning_label = visualizer.get_edges_related_label_info(G, G_extra_edge)

In [32]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from mpl_interactions import zoom_factory
from IPython.display import display
import matplotlib

from visualizer import CELL_EVENT_COLOR

plt.close('all')
matplotlib.use('TkAgg')  # Make sure this backend is compatible with your environment

frame = 0  # Initialize frame
max_frame = composer.frame_num - 1

# Assuming visualizer, composer, and G are defined
# Create the initial plot
plt.ioff()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10), gridspec_kw={'width_ratios': [5, 2]})
fig.tight_layout()


G_dict = {"manually link": G_manual, "New algo": G_new, "trackmate: LAP": G_lap, "trackmate: overlap": G_over}
G_list = sorted(G_dict)
g_index = 0
G_current = G_dict[G_list[g_index]]


labels = {}
tags = {}
poss = {}

for name, G in G_dict.items():
    label_info_1 = visualizer.get_label_info(G)
    label_info_2 = visualizer.get_generation_label_info(G)

    if name != 'manually link':
        G_extra_edge = set(G.edges()).difference(set(G_manual.edges()))
        G_manul_extra = set(G_manual.edges()).difference(set(G.edges()))
        warning_label = visualizer.get_edges_related_label_info(G, G_extra_edge)
        warning_label.update(visualizer.get_edges_related_label_info(G_manual, G_manul_extra))
        labels[name] = [label_info_1, label_info_2, warning_label]
    else:
        labels[name] = [label_info_1, label_info_2, label_info_1]
    tags[name] = visualizer.tag_type(G)
    poss[name] = visualizer.get_lineage_pos(G)


li_index = 0
label_info = labels[G_list[g_index]][li_index]
tag = tags[G_list[g_index]]
pos = poss[G_list[g_index]]


label_styles = ["regular", "circled", "empty"]
ls_index = 0
label_style = label_styles[ls_index]

cells_s = {CELL_EVENT_COLOR[CellEvent.BIRTH]: tag[CellEvent.BIRTH], CELL_EVENT_COLOR[CellEvent.DIE]: tag[CellEvent.DIE]}
edges_s = {CELL_EVENT_COLOR[CellEvent.SPLIT]: tag[CellEvent.SPLIT], CELL_EVENT_COLOR[CellEvent.MERGE]: tag[CellEvent.MERGE]}
        

image = composer.get_single_frame_phase(frame)
ax1 = visualizer.subplot_single_frame_phase(ax=ax1, G=G_current, image=image, cells_frame_dict=composer.cells_frame_dict, label_style  = label_style, frame=frame, info=label_info, fontsize=7, figsize=(15,15), representative_point=True)
ax1.set_title(f"Frame: {frame}. Algo:{G_list[g_index]}", weight = 600) 

ax2 = visualizer.subplot_lineage(ax2, G_current, pos, with_background=True, nodes_special=cells_s, edges_special=edges_s)
ax2.set_axis_off()
disconnect_zoom = zoom_factory(ax1)

# Update plot function
def update_plot(frame, ax1, ax2, fig, update_ax2=False):
    global label_styles, labels, li_index, ls_index, G_dict, G_list, g_index

    label_info = labels[G_list[g_index]][li_index]
    label_style = label_styles[ls_index]
    

    xlim = ax1.get_xlim()
    ylim = ax1.get_ylim()

    G_current = G_dict[G_list[g_index]]
    
    ax1.clear()
    image = composer.get_single_frame_phase(frame)
    ax1 = visualizer.subplot_single_frame_phase(ax=ax1, G=G_current, image=image, cells_frame_dict=composer.cells_frame_dict, label_style  = label_style, frame=frame, info=label_info, fontsize=7, figsize=(15,15), representative_point=True)
    ax1.set_xlim(xlim)
    ax1.set_ylim(ylim)
    ax1.set_title(f"Frame: {frame}. Algo:{G_list[g_index]}", weight = 600) 

    disconnect_zoom = zoom_factory(ax1)
    fig.canvas.draw_idle()

    if update_ax2 :
        ax2.clear()
        tag = tags[G_list[g_index]]
        pos = poss[G_list[g_index]]
        cells_s = {CELL_EVENT_COLOR[CellEvent.BIRTH]: tag[CellEvent.BIRTH], CELL_EVENT_COLOR[CellEvent.DIE]: tag[CellEvent.DIE]}
        edges_s = {CELL_EVENT_COLOR[CellEvent.SPLIT]: tag[CellEvent.SPLIT], CELL_EVENT_COLOR[CellEvent.MERGE]: tag[CellEvent.MERGE]}
        ax2 =  visualizer.subplot_lineage(ax2, G_current, pos, with_background=True, nodes_special=cells_s, edges_special=edges_s)
        ax2.set_axis_off()

    return ax1, ax2


def on_key(event):
    global frame, ax1, ax2, max_frame, fig, li_index , ls_index, g_index
    if event.key in ['right', 'down']:
        frame = min(frame + 1, max_frame) 
        update_plot(frame, ax1, ax2,  fig)
    elif event.key in ['left', 'up']:
        frame  = max(frame - 1, 0)
        update_plot(frame, ax1, ax2,  fig)
    elif event.key == 'c':
        ls_index += 1
        ls_index =  ls_index % len(label_styles)
        update_plot(frame, ax1, ax2,  fig)
    elif event.key == '1':
        li_index += 1
        li_index =  li_index % len(labels[G_list[0]])
        update_plot(frame, ax1, ax2, fig)
    elif event.key in ['g', 'G']:
        g_index  += 1
        g_index  =  g_index  % len(G_list)
        update_plot(frame, ax1, ax2, fig, update_ax2= True)
    

        

fig.canvas.mpl_connect('key_press_event', lambda event: on_key(event))

# Show the plot in a separate window
plt.show()

: 

In [20]:
G_silly = link_algorithm.silly_linker(composer)

In [21]:
G_dict['silly_algo'] = G_silly

In [31]:
for name, G in G_dict.items():
    correctness = len(set(G.edges()).intersection(set(G_manual.edges()))) * 1.0 /len(set(G.edges()).union(set(G_manual.edges())))
    print(f"{name}, {correctness:.4%}")


manually link, 100.0000%
New algo, 98.6318%
trackmate: LAP, 79.0360%
trackmate: overlap, 91.0168%
silly_algo, 98.6344%


In [30]:
defines = CellDefine.define_cells(G_manual)
manual_edges = set()

cells_s = set()
for define in defines:
    if not define.regular:
        cells_s.add(define.cell)

connected_nodes, connected_edges = visualizer.get_connected_edges_cells(G_manual, cells_s)
manual_edges.update(connected_edges)


for name, G in G_dict.items():
    defines = CellDefine.define_cells(G)
    edges_set = set()
    cells_s = set()
    for define in defines:
        if not define.regular:
         cells_s.add(define.cell)
    connected_nodes, connected_edges = visualizer.get_connected_edges_cells(G, cells_s)
    edges_set.update(connected_edges)

    correctness = len(edges_set.intersection(manual_edges)) * 1.0 /len(edges_set.union(manual_edges))
    print(f"{name}, {correctness:.4%}")


manually link, 100.0000%
New algo, 93.2540%
trackmate: LAP, 24.7878%
trackmate: overlap, 40.2827%
silly_algo, 91.7308%
